In [3]:
import numpy as np
import pandas as pd
import math
import copy
from ast import literal_eval
dataframe=pd.read_excel("./Datasets/COMP_SALE.xlsx")#provide the relative path to the xls file containing the data
#or just change the  pd.read_excel("") To pd.read_csv()
target='CAN BUY (CLASS LABEL)'#provide the Target class
dataframe.head()

,AGE,INCOME,STUDENT,CREDIT,CAN BUY (CLASS LABEL)
0,<30,HIGH,NO,FAIR,NO
1,<30,HIGH,NO,EXCELLENT,NO
2,31 - 40,HIGH,NO,FAIR,YES
3,>40,MEDIUM,NO,FAIR,YES
4,>40,LOW,YES,FAIR,YES


Next Cell contains the code to find the frquency of positve and negative target values for the categorical data

In [4]:
def frequency_finder(target=target,dataframe=dataframe):
    data_dict=dict(dataframe)
    T={}
    for v,k in enumerate(set(data_dict[target])):
       T[k]=v 
    frequency=dict.fromkeys(data_dict.keys())
    for k in data_dict.keys():
        frequency[k]=dict.fromkeys(set(data_dict[k]))
    for i in data_dict.keys():
        for j in frequency[i].keys():
            frequency[i][j]=[0,0]
    for i in range(len(dataframe)):
        temp=dict(dataframe.loc[i])
        for j in temp.keys():
            x=temp[j]
            frequency[j][x][T[temp[target]]]+=1
    return frequency
frequency_finder()

{'AGE': {'>40': [2, 3], '31 - 40': [0, 4], '<30': [3, 2]},
 'INCOME': {'MEDIUM': [2, 4], 'HIGH': [2, 2], 'LOW': [1, 3]},
 'STUDENT': {'NO': [4, 3], 'YES': [1, 6]},
 'CREDIT': {'FAIR': [2, 6], 'EXCELLENT': [3, 3]},
 'CAN BUY (CLASS LABEL)': {'NO': [5, 0], 'YES': [0, 9]}}

In [5]:
def Entropy(target=target,dataframe=dataframe):
    frequency=frequency_finder(target=target,dataframe=dataframe)
    entropy=copy.deepcopy(frequency) 
    for i in frequency.keys():
        for j in frequency[i].keys():
            p=frequency[i][j][0]
            n=frequency[i][j][1]
            s=p+n 
            if(p!=0) and (n!=0):
                entropy[i][j]=-(p/s*math.log(p/s,2))-(n/s*math.log(n/s,2))
            else:
                entropy[i][j]=0
    data_dict=dict(dataframe)
    T1={}
    for k,v in enumerate(set(data_dict[target])):
       T1[k]=v
    if 0 in T1.keys():
        p=float(sum(frequency[target][T1[0]]))
    else:
        p=0
    if 1 in T1.keys():
        n=float(sum(frequency[target][T1[1]]))
    else:
        n=0
    s=p+n
    if (p!=0)and(n!=0):
        entropy_target=-(p/s*math.log(p/s,2))-(n/s*math.log(n/s,2))
    else:
        entropy_target=0
    return entropy,entropy_target,s
Entropy()

({'AGE': {'>40': 0.9709505944546686, '31 - 40': 0, '<30': 0.9709505944546686},
  'INCOME': {'MEDIUM': 0.9182958340544896,
   'HIGH': 1.0,
   'LOW': 0.8112781244591328},
  'STUDENT': {'NO': 0.9852281360342516, 'YES': 0.5916727785823275},
  'CREDIT': {'FAIR': 0.8112781244591328, 'EXCELLENT': 1.0},
  'CAN BUY (CLASS LABEL)': {'NO': 0, 'YES': 0}},
 0.9402859586706309,
 14.0)

In [6]:
def gain(target=target,dataframe=dataframe):
    frequency=frequency_finder(target=target,dataframe=dataframe)
    entropy,e_target,length=Entropy(target=target,dataframe=dataframe)
    gain={}
    if e_target==0:
        return frequency,dataframe.loc[0][target],1
    for i in frequency.keys():
        s=0
        for j in frequency[i].keys():
            s+=(sum(frequency[i][j])/length)*entropy[i][j]
        gain[i]=e_target-s
    K,V='',0
    print(gain)
    for i in gain.keys():
        if i==target:
            continue
        if gain[i]> V:
            K=i
            V=gain[i]
    return frequency,K,0
gain()

{'AGE': 0.2467498197744391, 'INCOME': 0.029222565658954647, 'STUDENT': 0.15183550136234136, 'CREDIT': 0.04812703040826927, 'CAN BUY (CLASS LABEL)': 0.9402859586706309}


({'AGE': {'>40': [2, 3], '31 - 40': [0, 4], '<30': [3, 2]},
  'INCOME': {'MEDIUM': [2, 4], 'HIGH': [2, 2], 'LOW': [1, 3]},
  'STUDENT': {'NO': [4, 3], 'YES': [1, 6]},
  'CREDIT': {'FAIR': [2, 6], 'EXCELLENT': [3, 3]},
  'CAN BUY (CLASS LABEL)': {'NO': [5, 0], 'YES': [0, 9]}},
 'AGE',
 0)

In [7]:
def Tree(target=target,dataframe=dataframe):
    df=dataframe.copy()
    frequency,feature,end=gain(dataframe=df)
    tree={}
    if end==1:
        return feature
    tree[feature]=dict.fromkeys(frequency[feature].keys())
    for i in frequency[feature].keys():
        df=dataframe.copy()
        check=df[feature]==i
        df=df[check].reset_index()
        del df['index']
        df1=df.copy()
        df1.pop(feature)
        tree[feature][i]=Tree(dataframe=df1)
    return tree
Tree()

{'AGE': 0.2467498197744391, 'INCOME': 0.029222565658954647, 'STUDENT': 0.15183550136234136, 'CREDIT': 0.04812703040826927, 'CAN BUY (CLASS LABEL)': 0.9402859586706309}
{'INCOME': 0.01997309402197489, 'STUDENT': 0.01997309402197489, 'CREDIT': 0.9709505944546686, 'CAN BUY (CLASS LABEL)': 0.9709505944546686}
{'INCOME': 0.5709505944546686, 'STUDENT': 0.9709505944546686, 'CREDIT': 0.01997309402197489, 'CAN BUY (CLASS LABEL)': 0.9709505944546686}


{'AGE': {'>40': {'CREDIT': {'FAIR': 'YES', 'EXCELLENT': 'NO'}},
  '31 - 40': 'YES',
  '<30': {'STUDENT': {'NO': 'NO', 'YES': 'YES'}}}}

In [8]:
def get(input_data):
    try:
        return literal_eval(input_data)
    except (ValueError, SyntaxError):
        # A string, so return str
        return str

In [10]:
def Predict():
    tree=Tree(dataframe=dataframe)
    ip={}
    for i in list(dataframe.keys()):
        if i==target:
            continue
        j=input("Enter the "+i+" value")
        t=get(j)
        if(type(t)==type(2)):
            ip[i]=t
        else:
            ip[i]=j
    p=tree
    while(type(p)==type({})):
        x=list(p.keys())[0]
        t=ip[x]
        p=p[x][t]
    print("Prediction is =")
    print(p)
Predict()

{'AGE': 0.2467498197744391, 'INCOME': 0.029222565658954647, 'STUDENT': 0.15183550136234136, 'CREDIT': 0.04812703040826927, 'CAN BUY (CLASS LABEL)': 0.9402859586706309}
{'INCOME': 0.01997309402197489, 'STUDENT': 0.01997309402197489, 'CREDIT': 0.9709505944546686, 'CAN BUY (CLASS LABEL)': 0.9709505944546686}
{'INCOME': 0.5709505944546686, 'STUDENT': 0.9709505944546686, 'CREDIT': 0.01997309402197489, 'CAN BUY (CLASS LABEL)': 0.9709505944546686}


Enter the AGE value <30
Enter the INCOME value HIGH
Enter the STUDENT value NO
Enter the CREDIT value FAIR


Prediction is =
NO
